# **Text Classification**

In [ ]:
!pip install -q tensorflow_gpu==2.4.1

     |████████████████████████████████| 394.3MB 41kB/s 
     |████████████████████████████████| 471kB 48.4MB/s 
     |████████████████████████████████| 10.6MB 46.5MB/s 


In [ ]:
!pip install sentencepiece==0.1.91

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [ ]:
!pip install -q ktrain

In [ ]:
from sklearn.datasets import fetch_20newsgroups
import ktrain 
from ktrain import text

In [ ]:
def preprocess_dataset():
  classes = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']
  train_data = fetch_20newsgroups(subset='train', categories=classes, shuffle=True, random_state=42)
  test_data = fetch_20newsgroups(subset='test', categories=classes, shuffle=True, random_state=42)

  return train_data.data, train_data.target, test_data.data, test_data.target, classes

In [ ]:
def create_text_classification_model():
  MODEL_NAME = 'distilbert-base-uncased'
  train_features, train_labels, test_features, test_labels, train_classes = preprocess_dataset()
  trans = text.Transformer(MODEL_NAME, maxlen=500, classes=train_classes)
  train_preprocess = trans.preprocess_train(train_features, train_labels)
  val_preprocess = trans.preprocess_test(test_features, test_labels)
  model_data = trans.get_classifier()
  classification_model = ktrain.get_learner(model_data, train_data=train_preprocess, val_data=val_preprocess, batch_size=6)
  classification_model.fit_onecycle(5e-5, 4)
  return classification_model, trans

In [ ]:
def predict_category(classification_model, trans, input_text):
  predictor = ktrain.get_predictor(classification_model.model, preproc=trans)
  results = predictor.predict(input_text)
  return results

In [ ]:
classification_model, trans = create_text_classification_model()
input_text = 'Babies with down syndrome have an extra chromosome.'
predict(predict_category(classification_model, trans, input_text))

/usr/local/lib/python3.7/dist-packages/ktrain/text/preprocessor.py:414: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


preprocessing train...
language: en
train sequence lengths:
	mean : 308
	95percentile : 837
	99percentile : 1938


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 343
	95percentile : 979
	99percentile : 2562




begin training using onecycle policy with max lr of 5e-05...
Epoch 1/4
377/377 [==============================] - ETA: 0s - loss: 0.9501 - accuracy: 0.6615 

KeyboardInterrupt: ignored